In [1]:
### 🧠 What is Query Decomposition?
# Query decomposition is the process of taking a complex, multi-part question and 
# breaking it into simpler, atomic sub-questions that can each be retrieved and answered individually.

#### ✅ Why Use Query Decomposition?
# - Complex queries often involve multiple concepts
# - LLMs or retrievers may miss parts of the original question
# - It enables multi-hop reasoning (answering in steps)
# - Allows parallelism (especially in multi-agent frameworks)

In [2]:
from langchain.chat_models import init_chat_model
from langchain.prompts import PromptTemplate
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.runnables import RunnableSequence

c:\Users\hites\OneDrive\Desktop\AI\agentic-rag\query-enhancement\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Step 1: Load and embed the document
loader = TextLoader("data/langchain_crewai_dataset.txt")
docs = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
chunks = splitter.split_documents(docs)

embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(chunks, embedding)
retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 4, "lambda_mult": 0.7})

In [5]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

llm=init_chat_model(model="groq:gemma2-9b-it")

In [6]:
# Step 3: Query decomposition
decomposition_prompt = PromptTemplate.from_template("""
You are an AI assistant. Decompose the following complex question into 2 to 4 smaller sub-questions for better document retrieval.

Question: "{question}"

Sub-questions:
""")
decomposition_chain = decomposition_prompt | llm | StrOutputParser()

In [7]:
query = "How does LangChain use memory and agents compared to CrewAI?"
decomposition_question=decomposition_chain.invoke({"question": query})

In [8]:
print(decomposition_question)

Here are some sub-questions to decompose the complex question:

1. **What types of memory mechanisms does LangChain offer for its models?** 
2. **How do LangChain agents utilize memory to perform tasks?**
3. **What memory capabilities does CrewAI provide for its models?**
4. **How do CrewAI agents leverage memory in their decision-making processes?**


These sub-questions focus on specific aspects of memory and agent functionality in both LangChain and CrewAI, allowing for more focused document retrieval and comparison. 



In [9]:
# Step 4: QA chain per sub-question
qa_prompt = PromptTemplate.from_template("""
Use the context below to answer the question.

Context:
{context}

Question: {input}
""")
qa_chain = create_stuff_documents_chain(llm=llm, prompt=qa_prompt)

In [10]:
# Step 5: Full RAG pipeline logic
def full_query_decomposition_rag_pipeline(user_query):
    # Decompose the query
    sub_qs_text = decomposition_chain.invoke({"question": user_query})
    sub_questions = [q.strip("-•1234567890. ").strip() for q in sub_qs_text.split("\n") if q.strip()]
    
    results = []
    for subq in sub_questions:
        docs = retriever.invoke(subq)
        result = qa_chain.invoke({"input": subq, "context": docs})
        results.append(f"Q: {subq}\nA: {result}")
    
    return "\n\n".join(results)

In [11]:
# Step 6: Run
query = "How does LangChain use memory and agents compared to CrewAI?"
final_answer = full_query_decomposition_rag_pipeline(query)
print("✅ Final Answer:\n")
print(final_answer)

✅ Final Answer:

Q: Here are some sub-questions that break down the complex question:
A: Please provide the complex question you want to break down into sub-questions.  

I need the main question to help you formulate the sub-questions effectively. 😊 


Q: **What memory mechanisms does LangChain employ?**
A: LangChain uses memory modules like **ConversationBufferMemory** and **ConversationSummaryMemory**. 


These modules allow the LLM to:

* **Maintain awareness of previous conversation turns** (ConversationBufferMemory)
* **Summarize long interactions to fit within token limits** (ConversationSummaryMemory) 


Q: **How do LangChain agents utilize memory?**
A: According to the context, LangChain agents use **context-aware memory** across steps. 

This means they can remember and utilize information from previous steps in a sequence to inform their decisions and actions in subsequent steps. 


Q: **What are the memory capabilities of CrewAI?**
A: The provided context does not mention t